# 10.1 GroupBy Mechanics

Categorizing a dataset and applying a function to each group, whether an aggregation or transformation, is often a critical component of a data analysis workflow. After loading, merging, and preparing a dataset, you may need to compute group statistics or possibly pivot table for reporting or visualization purpose

- Split a pandas object into pieces using one or more keys (in the form of functions, arrays, or DataFrame column names)
- Calculate group summary statistics, like count, mean, or standard deviation, or a user-defined function
- Apply within-group transformations or other manipulations, like normalization, linear regression, rank, or subset selection
- Compute pivot tables and cross-tabulations
- Perform quantile analysis and other statistical group analyses

**split-apply-combine**: Group operations

- In the first stage of the process, data contained in a pandas object, whether a Series, DataFrame, or otherwise, is split into groups based on one or more keys that you provide. The splitting is performed on a particular axis of an object. For example, a DataFrame can be grouped on its rows (axis = 0) or its columns (axis = 1). 
- Once this is done, a function is applied to each group, producing a new value
- Finally, the results of all those function applications are combined into a result object. The form of the resulting object will usually depend on what's being done to the data

**Grouping keys:** Each grouping key can take many forms, and the keys do not have to be all of the same type:
- A list or array of values that is the same length as the axis being grouped
- A value indicating a column name in a DataFrame
- A dict or Series giving a correspondence between the values on the axis being grouped and the group names 
- A function to be invoked on the axis index or the individual labels in the index

In [1]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'key1':['a','a','b','b','a'],
                   'key2':['one','two','one','two','one'],
                   'data1':np.random.randn(5),
                   'data2':np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.077087,0.498347
1,a,two,-0.701938,-1.691719
2,b,one,-0.458730,-0.302157
3,b,two,-0.581331,1.295178
4,a,one,-0.544295,-0.930337


In [2]:
grouped = df['data1'].groupby(df['key1'])
grouped

The grouped variable is now a GroupBy object. It has not actually computed anything yet except for some intermediate data about the group key df['key1']. The idea is that this object has all of the information needed to then apply some operation to each of the groups. 

In [4]:
type(grouped)

pandas.core.groupby.generic.SeriesGroupBy

In [6]:
grouped.mean()

key1
a   -0.441107
b   -0.520030
Name: data1, dtype: float64

The data (a Series) has been aggregated according to the group key, producing a new Series that is now indexed by the unique values in the key1 column

Group the data using two keys, and the resulting Series now has a hierarchical index consisting of the unique pairs of keys observed

In [8]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.310691
      two    -0.701938
b     one    -0.458730
      two    -0.581331
Name: data1, dtype: float64

In [9]:
means.unstack()

key2,one,two
key1,,
a,-0.310691,-0.701938
b,-0.458730,-0.581331


In [18]:
states = np.array(['Ohio','California','California','Ohio','Ohio'])
years = np.array([2005,2005,2006,2005,2006])

In [19]:
states

array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'], dtype='<U10')

In [20]:
years

array([2005, 2005, 2006, 2005, 2006])

In [21]:
np.array(df['data1'])

array([-0.07708712, -0.70193788, -0.45872971, -0.58133094, -0.54429473])

In [22]:
df['data1'].groupby([states, years]).mean()

California  2005   -0.701938
            2006   -0.458730
Ohio        2005   -0.329209
            2006   -0.544295
Name: data1, dtype: float64

Frequently the grouping information is found in the same DataFrame as the data you want to work on. In that case, you can pass column names (whether those are strings, numbers, or other python objects) as the group keys 

In [23]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.441107,-0.707903
b,-0.520030,0.496510


In [24]:
df.groupby(df['key1']).mean()

,data1,data2
key1,,
a,-0.441107,-0.707903
b,-0.520030,0.496510


In [25]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.310691 -0.215995
     two  -0.701938 -1.691719
b    one  -0.458730 -0.302157
     two  -0.581331  1.295178

In [26]:
df.groupby([df['key1'], df['key2']]).mean()

data1     data2
key1 key2                    
a    one  -0.310691 -0.215995
     two  -0.701938 -1.691719
b    one  -0.458730 -0.302157
     two  -0.581331  1.295178

In [27]:
df.groupby(['key1', 'key2']).size() # return group sizes

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

## 10.1.1 Iterating Over Groups

The GroupBy object supports iteration, generating a sequence of 2-tuples containing the group name along with the chunk of data

In [29]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.077087  0.498347
1    a  two -0.701938 -1.691719
4    a  one -0.544295 -0.930337
b
  key1 key2     data1     data2
2    b  one -0.458730 -0.302157
3    b  two -0.581331  1.295178


In [39]:
df.groupby('key1').describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.441107  0.324955 -0.701938 -0.623116 -0.544295 -0.310691   
b      2.0 -0.520030  0.086692 -0.581331 -0.550681 -0.520030 -0.489380   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a    -0.077087   3.0 -0.707903  1.111848 -1.691719 -1.311028 -0.930337   
b    -0.458730   2.0  0.496510  1.129487 -0.302157  0.097176  0.496510   

                          
           75%       max  
key1                      
a    -0.215995  0.498347  
b     0.895844  1.295178

In [40]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.077087  0.498347
4    a  one -0.544295 -0.930337
('a', 'two')
  key1 key2     data1     data2
1    a  two -0.701938 -1.691719
('b', 'one')
  key1 key2    data1     data2
2    b  one -0.45873 -0.302157
('b', 'two')
  key1 key2     data1     data2
3    b  two -0.581331  1.295178


In [42]:
df.groupby(['key1', 'key2']).describe()

data1                                                              \
          count      mean       std       min       25%       50%       75%   
key1 key2                                                                     
a    one    2.0 -0.310691  0.330366 -0.544295 -0.427493 -0.310691 -0.193889   
     two    1.0 -0.701938       NaN -0.701938 -0.701938 -0.701938 -0.701938   
b    one    1.0 -0.458730       NaN -0.458730 -0.458730 -0.458730 -0.458730   
     two    1.0 -0.581331       NaN -0.581331 -0.581331 -0.581331 -0.581331   

                    data2                                                    \
                max count      mean       std       min       25%       50%   
key1 key2                                                                     
a    one  -0.077087   2.0 -0.215995  1.010232 -0.930337 -0.573166 -0.215995   
     two  -0.701938   1.0 -1.691719       NaN -1.691719 -1.691719 -1.691719   
b    one  -0.458730   1.0 -0.302157       NaN -0.302157 -0.302157 -0.302157   
     two  -0.581331   1.0  1.295178       NaN  1.295178  1.295178  1.295178   

                               
                75%       max  
key1 key2                      
a    one   0.141176  0.498347  
     two  -1.691719 -1.691719  
b    one  -0.302157 -0.302157  
     two   1.295178  1.295178

In [43]:
pieces = dict(list(df.groupby('key1')))
pieces

{'a':   key1 key2     data1     data2
 0    a  one -0.077087  0.498347
 1    a  two -0.701938 -1.691719
 4    a  one -0.544295 -0.930337,
 'b':   key1 key2     data1     data2
 2    b  one -0.458730 -0.302157
 3    b  two -0.581331  1.295178}

In [44]:
pieces['a']

,key1,key2,data1,data2
0,a,one,-0.077087,0.498347
1,a,two,-0.701938,-1.691719
4,a,one,-0.544295,-0.930337


In [45]:
pieces['b']

,key1,key2,data1,data2
2,b,one,-0.458730,-0.302157
3,b,two,-0.581331,1.295178


By default, groupby groups on axis = 0. But you can group on any of the other axes

In [46]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [47]:
grouped = df.groupby(df.dtypes, axis = 1)
grouped

In [48]:
dict(list(grouped))

{dtype('float64'):       data1     data2
 0 -0.077087  0.498347
 1 -0.701938 -1.691719
 2 -0.458730 -0.302157
 3 -0.581331  1.295178
 4 -0.544295 -0.930337,
 dtype('O'):   key1 key2
 0    a  one
 1    a  two
 2    b  one
 3    b  two
 4    a  one}

In [49]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.077087  0.498347
1 -0.701938 -1.691719
2 -0.458730 -0.302157
3 -0.581331  1.295178
4 -0.544295 -0.930337
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


## 10.1.2 Selecting a Column or Subset of Columns

Indexing a GroupBy object created from a DataFrame with a column name or array of column names has the effect of column subsetting for aggregation

In [51]:
df

,key1,key2,data1,data2
0,a,one,-0.077087,0.498347
1,a,two,-0.701938,-1.691719
2,b,one,-0.458730,-0.302157
3,b,two,-0.581331,1.295178
4,a,one,-0.544295,-0.930337


In [52]:
dict(list(df.groupby('key1')))

{'a':   key1 key2     data1     data2
 0    a  one -0.077087  0.498347
 1    a  two -0.701938 -1.691719
 4    a  one -0.544295 -0.930337,
 'b':   key1 key2     data1     data2
 2    b  one -0.458730 -0.302157
 3    b  two -0.581331  1.295178}

In [53]:
dict(list(df.groupby('key1')['data1']))

{'a': 0   -0.077087
 1   -0.701938
 4   -0.544295
 Name: data1, dtype: float64,
 'b': 2   -0.458730
 3   -0.581331
 Name: data1, dtype: float64}

In [54]:
df.groupby('key1')['data1']

In [55]:
df['data1'].groupby(df['key1'])

In [56]:
df.groupby('key1')[['data2']]

In [57]:
df[['data2']].groupby(df['key1'])

Especially for large datasets, it may be desirable to aggregate only a few columns

In [64]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.215995
     two  -1.691719
b    one  -0.302157
     two   1.295178

In [65]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.310691 -0.215995
     two  -0.701938 -1.691719
b    one  -0.458730 -0.302157
     two  -0.581331  1.295178

In [66]:
df.groupby(['key1', 'key2'])['data2'].mean()

key1  key2
a     one    -0.215995
      two    -1.691719
b     one    -0.302157
      two     1.295178
Name: data2, dtype: float64

In [67]:
s_grouped = df.groupby(['key1','key2'])['data2']
s_grouped

In [68]:
s_grouped.mean()

key1  key2
a     one    -0.215995
      two    -1.691719
b     one    -0.302157
      two     1.295178
Name: data2, dtype: float64

## 10.1.3 Grouping with Dicts and Series

Grouping information may exist in a form other than an array

In [69]:
people = pd.DataFrame(np.random.randn(5,5),
                      columns = ['a','b','c','d','e'],
                      index = ['Joe','Steve','Wes','Jim','Travis'])
people

,a,b,c,d,e
Joe,-0.665759,-1.285037,-0.578495,1.810249,-0.397001
Steve,-1.440291,0.602391,-1.718124,-0.892897,-0.490787
Wes,-1.473977,0.487358,-0.509579,1.395556,-0.042165
Jim,2.882766,-1.337231,0.290871,-0.578271,1.717435
Travis,-0.807716,-1.121798,-0.189297,1.190855,-1.081513


In [70]:
people.iloc[2:3,[1,2]] = np.nan
people

,a,b,c,d,e
Joe,-0.665759,-1.285037,-0.578495,1.810249,-0.397001
Steve,-1.440291,0.602391,-1.718124,-0.892897,-0.490787
Wes,-1.473977,NaN,NaN,1.395556,-0.042165
Jim,2.882766,-1.337231,0.290871,-0.578271,1.717435
Travis,-0.807716,-1.121798,-0.189297,1.190855,-1.081513


In [71]:
mapping = {'a':'red','b':'red','c':'blue','d':'blue','e':'red','f':'orange'}

In [72]:
by_column = people.groupby(mapping, axis = 1)
by_column

In [73]:
dict(list(by_column))

{'blue':                c         d
 Joe    -0.578495  1.810249
 Steve  -1.718124 -0.892897
 Wes          NaN  1.395556
 Jim     0.290871 -0.578271
 Travis -0.189297  1.190855,
 'red':                a         b         e
 Joe    -0.665759 -1.285037 -0.397001
 Steve  -1.440291  0.602391 -0.490787
 Wes    -1.473977       NaN -0.042165
 Jim     2.882766 -1.337231  1.717435
 Travis -0.807716 -1.121798 -1.081513}

In [74]:
by_column.sum()

,blue,red
Joe,1.231754,-2.347796
Steve,-2.611021,-1.328686
Wes,1.395556,-1.516142
Jim,-0.287400,3.262971
Travis,1.001557,-3.011026


The same functionality holds for Series, which can be viewed as a fixed-size mapping

In [75]:
mapping

{'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}

In [76]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [77]:
people.groupby(map_series, axis = 1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


## 10.1.4 Grouping with Functions

Using python functions is a more generic way of defining a group mapping compared with a dict or Series. Any function passed as a group key will be called once per index value, with the return values being used as the group names


In [78]:
people

,a,b,c,d,e
Joe,-0.665759,-1.285037,-0.578495,1.810249,-0.397001
Steve,-1.440291,0.602391,-1.718124,-0.892897,-0.490787
Wes,-1.473977,NaN,NaN,1.395556,-0.042165
Jim,2.882766,-1.337231,0.290871,-0.578271,1.717435
Travis,-0.807716,-1.121798,-0.189297,1.190855,-1.081513


In [81]:
people.groupby(len).sum() # group by the len of the row indices then sum the values 

,a,b,c,d,e
3,0.743031,-2.622268,-0.287624,2.627533,1.278269
5,-1.440291,0.602391,-1.718124,-0.892897,-0.490787
6,-0.807716,-1.121798,-0.189297,1.190855,-1.081513


In [82]:
people.groupby(len)

In [83]:
dict(list(people.groupby(len)))

{3:             a         b         c         d         e
 Joe -0.665759 -1.285037 -0.578495  1.810249 -0.397001
 Wes -1.473977       NaN       NaN  1.395556 -0.042165
 Jim  2.882766 -1.337231  0.290871 -0.578271  1.717435,
 5:               a         b         c         d         e
 Steve -1.440291  0.602391 -1.718124 -0.892897 -0.490787,
 6:                a         b         c         d         e
 Travis -0.807716 -1.121798 -0.189297  1.190855 -1.081513}

Mixing functions with arrays, dict, or Series is not a problem as everything gets converted to arrays internally

In [84]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -1.473977 -1.285037 -0.578495  1.395556 -0.397001
  two  2.882766 -1.337231  0.290871 -0.578271  1.717435
5 one -1.440291  0.602391 -1.718124 -0.892897 -0.490787
6 two -0.807716 -1.121798 -0.189297  1.190855 -1.081513

## 10.1.5 Grouping by Index Levels

A final convenience for hierarchically indexed datasets is the ability to aggregate using one of the levels of an axis index

In [85]:
columns = pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],
                                     [1,3,5,1,3]],
                                    names = ['cty','tenor'])
columns

MultiIndex([('US', 1),
            ('US', 3),
            ('US', 5),
            ('JP', 1),
            ('JP', 3)],
           names=['cty', 'tenor'])

In [86]:
hier_df = pd.DataFrame(np.random.randn(4,5), columns = columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.302112 -0.217275 -0.086066 -1.494482  1.363796
1      0.955466  0.785777 -0.674572 -0.043797 -0.279073
2      0.801789  0.743909  0.309915 -1.210449 -0.502948
3     -1.251386  1.408161  0.304833  0.429884 -0.256017

In [87]:
hier_df.groupby(level = 'cty', axis = 1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [88]:
hier_df.groupby(level = 0, axis = 1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [89]:
hier_df.groupby(level = 'tenor', axis = 1).count()

tenor,1,3,5
0,2,2,1
1,2,2,1
2,2,2,1
3,2,2,1


In [91]:
hier_df.groupby(level = 1, axis = 1).count()

tenor,1,3,5
0,2,2,1
1,2,2,1
2,2,2,1
3,2,2,1


# 10.2 Data Aggregation

Aggregations refer to any data transformation that produces scalar values from arrays. Examples are mean, count, min and sum

In [92]:
df

,key1,key2,data1,data2
0,a,one,-0.077087,0.498347
1,a,two,-0.701938,-1.691719
2,b,one,-0.458730,-0.302157
3,b,two,-0.581331,1.295178
4,a,one,-0.544295,-0.930337


In [93]:
grouped = df.groupby('key1')
grouped

In [94]:
dict(list(grouped))

{'a':   key1 key2     data1     data2
 0    a  one -0.077087  0.498347
 1    a  two -0.701938 -1.691719
 4    a  one -0.544295 -0.930337,
 'b':   key1 key2     data1     data2
 2    b  one -0.458730 -0.302157
 3    b  two -0.581331  1.295178}

In [95]:
grouped['data1'].quantile(0.9)

key1
a   -0.170529
b   -0.470990
Name: data1, dtype: float64

In [96]:
grouped['data1'].quantile(0.5)

key1
a   -0.544295
b   -0.520030
Name: data1, dtype: float64

In [97]:
grouped['data1'].median()

key1
a   -0.544295
b   -0.520030
Name: data1, dtype: float64

To use your own aggregation functions, pass any function that aggregates an array to the aggregate or agg method

In [98]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [99]:
grouped.agg(func = peak_to_peak)

,data1,data2
key1,,
a,0.624851,2.190067
b,0.122601,1.597335


In [100]:
grouped.agg(func = min)

,key2,data1,data2
key1,,,
a,one,-0.701938,-1.691719
b,one,-0.581331,-0.302157


In [101]:
grouped.min()

,key2,data1,data2
key1,,,
a,one,-0.701938,-1.691719
b,one,-0.581331,-0.302157


In [102]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.441107  0.324955 -0.701938 -0.623116 -0.544295 -0.310691   
b      2.0 -0.520030  0.086692 -0.581331 -0.550681 -0.520030 -0.489380   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a    -0.077087   3.0 -0.707903  1.111848 -1.691719 -1.311028 -0.930337   
b    -0.458730   2.0  0.496510  1.129487 -0.302157  0.097176  0.496510   

                          
           75%       max  
key1                      
a    -0.215995  0.498347  
b     0.895844  1.295178

In [103]:
grouped.describe().stack()

data1     data2
key1                          
a    count  3.000000  3.000000
     mean  -0.441107 -0.707903
     std    0.324955  1.111848
     min   -0.701938 -1.691719
     25%   -0.623116 -1.311028
     50%   -0.544295 -0.930337
     75%   -0.310691 -0.215995
     max   -0.077087  0.498347
b    count  2.000000  2.000000
     mean  -0.520030  0.496510
     std    0.086692  1.129487
     min   -0.581331 -0.302157
     25%   -0.550681  0.097176
     50%   -0.520030  0.496510
     75%   -0.489380  0.895844
     max   -0.458730  1.295178

Custom aggregation functions are generally much slower than the optimized functions. This is because there is some extra overhead (function calls, data rearrangement) in constructing the intermediate group data chunks

## 10.2.1 Column-Wise and Multiple Function Application

In [104]:
tips = pd.read_csv('/Users/boyuan/Desktop/OneDrive/Python for data analysis 2nd/examples/tips.csv')
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [105]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


Aggregating a Series or all of the columns of a DataFrame is a matter of using aggregate with the desired function or calling a method like mean or std. However, you may want to aggregate using a different function depending on the column, or multiple functions at once.

In [106]:
grouped = tips.groupby(['day', 'smoker'])
grouped

In [107]:
grouped_pct = grouped['tip_pct']

In [108]:
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [110]:
grouped_pct.mean()

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

If you pass a list of functions or function names instead, you get back a DataFrame with column names taken from the functions

In [111]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

Pass tuple (name, function). If you pass a list of (name, function) tuples, the first element of each tuple will be used as the DataFrame column names (you can think of a list of 2-tuples as an ordered mapping)

In [116]:
grouped_pct.agg([('foo', 'mean'),
                 ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

With a DataFrame you have more options, as you can specify a list of functions to apply to all of the columns or different functions per column. 

In [118]:
functions = ['count', 'mean', 'max']
result = grouped[['tip_pct', 'total_bill']].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

The resulting DataFrame has hierarchical columns, the same as you would get aggregating each column separately and using concat to glue the results together using the column names as the keys argument

In [119]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

A list of tuples with custom names can be passed

In [120]:
ftuples = [('Durchschnitt', 'mean'), ('Abwerchung', np.var)]
grouped[['tip_pct', 'total_bill']].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abwerchung Durchschnitt  Abwerchung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

Apply potentially different functions to one or more of the columns. Pass a dict to agg that contains a mapping of column names to any of the function specifications listed so far

In [121]:
grouped.agg({'tip': np.max,
             'size': 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [122]:
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'],
             'size': 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

In [124]:
grouped.agg({'tip_pct': [('f1', 'min'), ('f2', 'max'), ('f3', 'mean'), ('f4', 'std')],
             'size': [('f5', 'sum')]})

tip_pct                               size
                   f1        f2        f3        f4   f5
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

## 10.2.2 Returning Aggregated Data Without Row Indexes 

The aggregated data comes back with an index, potentially hierarchical, composed from the unique group key combinations. Since this isn't always desirable, you can disable this behavior in most cases by passing as_index = False to groupby

In [126]:
tips.groupby(['day', 'smoker'], as_index = True).mean()

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

In [127]:
tips.groupby(['day', 'smoker'], as_index = False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [128]:
tips.groupby(['day', 'smoker'], as_index = True).mean().reset_index()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


Using the as_index = False method avoids some unnecessary computations than using reset_index on the result

# 10.3 Apply: General split-apply-combine

apply splits the object being manipulated into pieces, invokes the passed function on each piece, and then attempts to concatenate the pieces together

In [130]:
# select the top five tip_pct values by group

def top(df, n = 5, column = 'tip_pct'):
    return df.sort_values(by = column)[-n:]
top(tips, n = 6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [131]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

The top function is called on each row group from the DataFrame, and then the results are glued together using pandas.concat, labeling the pieces with the group names. The result therefore has a hierarchical index whose inner level contains index values from the original DataFrame

If you pass a function to apply that takes other arguments or keywords, you can pass these after the function

In [132]:
tips.groupby(['smoker', 'day']).apply(top, n = 1, column = 'total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [133]:
tips.groupby(['smoker', 'day']).apply(top, n = 2, column = 'total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  91        22.49   3.50     No   Fri  Dinner     2  0.155625
            94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  59        48.27   6.73     No   Sat  Dinner     4  0.139424
            212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  112       38.07   4.00     No   Sun  Dinner     3  0.105070
            156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 85        34.83   5.17     No  Thur   Lunch     4  0.148435
            142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  90        28.97   3.00    Yes   Fri  Dinner     2  0.103555
            95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  102       44.30   2.50    Yes   Sat  Dinner     3  0.056433
            170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  184       40.55   3.00    Yes   Sun  Dinner     2  0.073983
            182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 83        32.68   5.00    Yes  Thur   Lunch     2  0.152999
            197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [135]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [136]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

Inside GroupBy, when you invoke a method like describe, it is actually just a shortcut 

In [137]:
f = lambda x: x.describe()
grouped.apply(f)

total_bill       tip  size   tip_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

In [138]:
grouped.apply(lambda x: x.describe())

total_bill       tip  size   tip_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

## 10.3.1 Suppressing the Group Keys

In [139]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [114]:
tips.groupby('smoker', group_keys = False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


## 10.3.2 Quantile and Bucket Analysis

pandas has some tools, in particular cut and qcut, for slicing data up into buckets with bins of your choosing or by sample quantiles. Combining these functions with groupby makes it convenient to perform bucket or quantile analysis on a dataset. 

In [140]:
frame = pd.DataFrame({'data1':np.random.randn(1000),
                      'data2':np.random.randn(1000)})
frame

,data1,data2
0,0.744860,-0.957063
1,0.633025,-0.460320
2,0.001015,0.745618
3,0.274356,-0.617051
4,-0.965390,0.589038
...,...,...
995,0.401807,-1.263741
996,0.611757,0.401283
997,0.043816,0.394816
998,-3.150567,1.611428


In [141]:
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0      (-0.106, 1.49]
1      (-0.106, 1.49]
2      (-0.106, 1.49]
3      (-0.106, 1.49]
4    (-1.702, -0.106]
5    (-1.702, -0.106]
6      (-0.106, 1.49]
7      (-0.106, 1.49]
8      (-0.106, 1.49]
9    (-1.702, -0.106]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.305, -1.702] < (-1.702, -0.106] < (-0.106, 1.49] < (1.49, 3.086]]

In [142]:
quartiles.value_counts()

(-0.106, 1.49]      452
(-1.702, -0.106]    449
(1.49, 3.086]        61
(-3.305, -1.702]     38
Name: data1, dtype: int64

The Categorical object returned by cut can be passed directly to groupby

In [143]:
def get_stats(group):
    return {'min': group.min(),
            'max': group.max(),
            'count': group.count(),
            'mean': group.mean()}

grouped = frame.data2.groupby(quartiles)
grouped

In [144]:
grouped.apply(get_stats)

data1                  
(-3.305, -1.702]  min       -2.238639
                  max        1.695643
                  count     38.000000
                  mean      -0.006969
(-1.702, -0.106]  min       -2.721730
                  max        2.998580
                  count    449.000000
                  mean       0.001818
(-0.106, 1.49]    min       -3.379533
                  max        3.390108
                  count    452.000000
                  mean      -0.013595
(1.49, 3.086]     min       -1.699552
                  max        2.565332
                  count     61.000000
                  mean       0.155332
Name: data2, dtype: float64

In [145]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.305, -1.702]",-2.238639,1.695643,38.0,-0.006969
"(-1.702, -0.106]",-2.721730,2.998580,449.0,0.001818
"(-0.106, 1.49]",-3.379533,3.390108,452.0,-0.013595
"(1.49, 3.086]",-1.699552,2.565332,61.0,0.155332


In [146]:
grouping = pd.qcut(frame.data1, 10, labels = False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats)

data1       
0      min       -2.238639
       max        2.520116
       count    100.000000
       mean      -0.031795
1      min       -2.721730
       max        2.254216
       count    100.000000
       mean       0.022826
2      min       -1.953043
       max        2.998580
       count    100.000000
       mean       0.021993
3      min       -2.668678
       max        1.911172
       count    100.000000
       mean      -0.014361
4      min       -2.431310
       max        2.800951
       count    100.000000
       mean      -0.026923
5      min       -3.379533
       max        3.390108
       count    100.000000
       mean      -0.031379
6      min       -2.194237
       max        1.905320
       count    100.000000
       mean      -0.125363
7      min       -2.873402
       max        2.436308
       count    100.000000
       mean       0.155191
8      min       -2.224733
       max        3.381333
       count    100.000000
       mean       0.054031
9      min     

In [148]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.238639,2.520116,100.0,-0.031795
1,-2.721730,2.254216,100.0,0.022826
2,-1.953043,2.998580,100.0,0.021993
3,-2.668678,1.911172,100.0,-0.014361
4,-2.431310,2.800951,100.0,-0.026923
5,-3.379533,3.390108,100.0,-0.031379
6,-2.194237,1.905320,100.0,-0.125363
7,-2.873402,2.436308,100.0,0.155191
8,-2.224733,3.381333,100.0,0.054031


## 10.3.3 Example: Filling Missing Values with Group-Specific Values

In [149]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1    0.762641
2         NaN
3   -0.822887
4         NaN
5   -0.145391
dtype: float64

In [150]:
s.fillna(s.mean())

0   -0.068546
1    0.762641
2   -0.068546
3   -0.822887
4   -0.068546
5   -0.145391
dtype: float64

Suppose you need the fill value to vary by group. One way to do this is to group the data and use apply with a function that calls fillna on each data chunk

In [151]:
states = ['Ohio','New York','Vermont','Florida','Oregon','Nevada','California','Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index = states)
data

Ohio          0.784052
New York     -0.027280
Vermont       0.467623
Florida      -0.344379
Oregon        0.049624
Nevada        0.734406
California   -0.842319
Idaho        -1.175136
dtype: float64

In [152]:
data[['Vermont','Nevada','Idaho']] = np.nan
data

Ohio          0.784052
New York     -0.027280
Vermont            NaN
Florida      -0.344379
Oregon        0.049624
Nevada             NaN
California   -0.842319
Idaho              NaN
dtype: float64

In [153]:
data.groupby(group_key).mean()

East    0.137464
West   -0.396348
dtype: float64

In [154]:
fill_mean = lambda x: x.fillna(x.mean())

data.groupby(group_key).apply(fill_mean)

Ohio          0.784052
New York     -0.027280
Vermont       0.137464
Florida      -0.344379
Oregon        0.049624
Nevada       -0.396348
California   -0.842319
Idaho        -0.396348
dtype: float64

In another case, you might have predefined fill values in your code that vary by group

In [155]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda x: x.fillna(fill_values[x.name])

data.groupby(group_key).apply(fill_func)

Ohio          0.784052
New York     -0.027280
Vermont       0.500000
Florida      -0.344379
Oregon        0.049624
Nevada       -1.000000
California   -0.842319
Idaho        -1.000000
dtype: float64

## 10.3.4 Example: Random Sampling and Permutation

In [156]:
suits = ['H','S','C','D']
card_val = (list(range(1,11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2,11)) + ['J','K','Q']
cards = []
for suit in ['H','S','C','D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index = cards)

In [157]:
deck

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
8S      8
9S      9
10S    10
JS     10
KS     10
QS     10
AC      1
2C      2
3C      3
4C      4
5C      5
6C      6
7C      7
8C      8
9C      9
10C    10
JC     10
KC     10
QC     10
AD      1
2D      2
3D      3
4D      4
5D      5
6D      6
7D      7
8D      8
9D      9
10D    10
JD     10
KD     10
QD     10
dtype: int64

In [158]:
len(deck)

52

In [159]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [160]:
# draw a hand of five cards from the deck 

def draw(deck, n = 5):
    return deck.sample(n)
draw(deck)

QC     10
5H      5
6H      6
3C      3
10H    10
dtype: int64

In [161]:
# two random cards from each suit 
get_suit = lambda x: x[-1] # last letter is suit
deck.groupby(get_suit).apply(draw, n = 2)

C  4C     4
   7C     7
D  AD     1
   2D     2
H  KH    10
   6H     6
S  AS     1
   8S     8
dtype: int64

In [162]:
deck.groupby(get_suit, group_keys = False).apply(draw, n = 2)

QC    10
3C     3
QD    10
JD    10
KH    10
AH     1
8S     8
AS     1
dtype: int64

## 10.3.5 Example: Group Weighted Average and Correlation

Under the split-apply-combine paradigm of groupby, operations between columns in a DataFrame or two Series, such as a group weighted average, are possible

In [163]:
df = pd.DataFrame({'category':['a','a','a','a','b','b','b','b'],
                   'data':np.random.randn(8),
                   'weights':np.random.rand(8)})
df

,category,data,weights
0,a,-0.650948,0.883466
1,a,-0.598630,0.211120
2,a,-2.647885,0.531425
3,a,-2.035180,0.330259
4,b,1.022488,0.127792
5,b,-0.161236,0.846924
6,b,1.342184,0.116341
7,b,1.097845,0.055242


In [164]:
grouped = df.groupby('category')

get_wavg = lambda x: np.average(x['data'], weights = x['weights'])

grouped.apply(get_wavg)

category
a   -1.421461
b    0.183991
dtype: float64

In [165]:
close_px = pd.read_csv('/Users/boyuan/Desktop/OneDrive/Python for data analysis 2nd/examples/stock_px_2.csv',
                       parse_dates = True,
                       index_col = 0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [166]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [167]:
# compute a DataFrame consisting of the yearly correlations of daily returns with SPX
spx_corr = lambda x: x.corrwith(x['SPX'])

rets = close_px.pct_change().dropna()

get_year = lambda x: x.year

by_year = rets.groupby(get_year)

by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [168]:
by_year.apply(lambda x: x['AAPL'].corr(x['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

## 10.3.6 Example: Group-Wise Linear Regression 

In [169]:
# executes an ordinary least squares (OLS) regression on each chunk of data

import statsmodels.api as sm

def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1
    result = sm.OLS(Y, X).fit()
    return result.params

In [170]:
by_year.apply(regress, yvar = 'AAPL', xvars = ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


# 10.4 Pivot Tables and Cross-Tabulation

A pivot table is a data summarization tool frequently found in spreadsheet programs and other data analysis software. It aggregates a table of data by one or more keys, arranging the data in a rectangle with some of the group keys along the rows and some along the columns. Pivot tables in Python with pandas are made possible through the groupby facility described in this chapter combined with reshape operations utilizing hierarchical indexing. DataFrame has a pivot_table method, and there is also a top-level pandas.pivot_table function. In addition to providing a convenience interface to groupby, pivot_table can add partial totals, also known as margins.

In [171]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


Default pivot_table aggregation type is group means

In [172]:
tips.pivot_table(index = ['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [173]:
tips.pivot_table(index = ['day', 'smoker'], aggfunc = 'mean')

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [175]:
tips.groupby(['day', 'smoker']).agg('mean')

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

In [177]:
tips.pivot_table(values = ['tip_pct', 'size'],
                 index = ['time', 'day'],
                 columns = 'smoker',
                 aggfunc = 'mean')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [178]:
tips.pivot_table(values = ['tip_pct','size'],
                 index = ['time', 'day'],
                 columns = 'smoker',
                 aggfunc = 'mean',
                 margins = True) # include partial totals by passing margins = True

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [180]:
tips.pivot_table(values = 'tip_pct', 
                 index = ['time','smoker'], 
                 columns = 'day', 
                 aggfunc = len,
                 margins = True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [181]:
tips.pivot_table(values = 'tip_pct', 
                 index = ['time','smoker'], 
                 columns = 'day', 
                 aggfunc = 'mean',
                 margins = True)

day                 Fri       Sat       Sun      Thur       All
time   smoker                                                  
Dinner No      0.139622  0.158048  0.160113  0.159744  0.158653
       Yes     0.165347  0.147906  0.187250       NaN  0.160828
Lunch  No      0.187735       NaN       NaN  0.160311  0.160920
       Yes     0.188937       NaN       NaN  0.163863  0.170404
All            0.169913  0.153152  0.166897  0.161276  0.160803

In [182]:
tips.pivot_table(values = 'tip_pct', 
                 index = ['time','size','smoker'], 
                 columns = 'day', 
                 aggfunc = 'mean', 
                 fill_value = 0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

## 10.4.1 Cross-Tabulations: Crosstab

A cross-tabulation (or crosstab) is a special case of a pivot table that computes group frequencies

In [183]:
data = pd.DataFrame({'Sample':[1,2,3,4,5,6,7,8,9,10],
                     'Nationality':['USA','Japan','USA','Japan','Japan','Japan','USA','USA','Japan','USA'],
                     'Handedness':['Right','Left','Right','Right','Left','Right','Right','Left','Right','Right']})
data

,Sample,Nationality,Handedness
0,1,USA,Right
1,2,Japan,Left
2,3,USA,Right
3,4,Japan,Right
4,5,Japan,Left
5,6,Japan,Right
6,7,USA,Right
7,8,USA,Left
8,9,Japan,Right
9,10,USA,Right


In [184]:
pd.crosstab(index = data.Nationality, columns = data.Handedness, margins = True)

Handedness,Left,Right,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [185]:
pd.crosstab(index = [tips.time, tips.day],
            columns = tips.smoker,
            margins = True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

# 10.5 Conclusion